In [21]:
# Add project root to Python path
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

print(f"✓ Project root: {project_root}")

✓ Project root: c:\Users\AUC\Desktop\Thesis\Quantum-Optimization-In-AP-Selection


In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set global scale factor for all plot elements
SCALE_FACTOR = 1.0  # Adjust this to scale all elements (e.g., 1.5 = 150%, 2.0 = 200%)

# Set publication-quality plotting defaults
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = int(12 * SCALE_FACTOR)
plt.rcParams['axes.linewidth'] = 2 * SCALE_FACTOR
plt.rcParams['lines.linewidth'] = 3 * SCALE_FACTOR
plt.rcParams['lines.markersize'] = 8 * SCALE_FACTOR
plt.rcParams['xtick.major.width'] = 2 * SCALE_FACTOR
plt.rcParams['ytick.major.width'] = 2 * SCALE_FACTOR
plt.rcParams['xtick.major.size'] = 6 * SCALE_FACTOR
plt.rcParams['ytick.major.size'] = 6 * SCALE_FACTOR
plt.rcParams['grid.linewidth'] = 1.5 * SCALE_FACTOR
plt.rcParams['legend.fontsize'] = int(11 * SCALE_FACTOR)
plt.rcParams['axes.labelsize'] = int(13 * SCALE_FACTOR)
plt.rcParams['axes.titlesize'] = int(14 * SCALE_FACTOR)

sns.set_style('whitegrid')

print("✓ All libraries imported successfully")
print(f"✓ Publication-quality plotting configured (DPI=300, Scale={SCALE_FACTOR}x)")

✓ All libraries imported successfully
✓ Publication-quality plotting configured (DPI=300, Scale=1.0x)


In [23]:
# Define paths
results_dir = project_root / 'data' / 'results'
phase1_file = results_dir / 'phase1_openjij_parameters.xlsx'
phase2_file = results_dir / 'phase2_openjij_parameters.xlsx'
pipeline_file = results_dir / 'pipeline_experiment_results.csv'
output_dir = results_dir / 'visualizations' / 'paper'
output_dir.mkdir(parents=True, exist_ok=True)

print("Loading benchmark results...")

# Load Phase 1 results
if not phase1_file.exists():
    raise FileNotFoundError(f"Phase 1 file not found: {phase1_file}")
phase1_df = pd.read_excel(phase1_file)
print(f"✓ Loaded Phase 1 results: {len(phase1_df)} configurations")

# Load Phase 2 results
if not phase2_file.exists():
    raise FileNotFoundError(f"Phase 2 file not found: {phase2_file}")
phase2_df = pd.read_excel(phase2_file)
print(f"✓ Loaded Phase 2 results: {len(phase2_df)} configurations")

# Load importance metrics comparison results
if not pipeline_file.exists():
    raise FileNotFoundError(f"Pipeline results file not found: {pipeline_file}")
pipeline_df = pd.read_csv(pipeline_file)
print(f"✓ Loaded pipeline results: {len(pipeline_df)} configurations")

print(f"\n✓ Output directory: {output_dir}")

Loading benchmark results...
✓ Loaded Phase 1 results: 96 configurations
✓ Loaded Phase 2 results: 144 configurations
✓ Loaded pipeline results: 5 configurations

✓ Output directory: c:\Users\AUC\Desktop\Thesis\Quantum-Optimization-In-AP-Selection\data\results\visualizations\paper


In [24]:
pipeline_df.head()

,Importance_Method,Num_APs,Selected_APs,Mean_3D_Error_m,Median_3D_Error_m,Min_Error_m,Max_Error_m,Floor_Accuracy_0,Floor_Accuracy_1,Floor_Accuracy_2
0,ENTROPY,20,"WAP016, WAP037, WAP090, WAP091, WAP103, WAP104...",16.997920,11.578227,0.206596,75.044313,0.661238,0.983713,1.000000
1,VARIANCE,20,"WAP089, WAP090, WAP091, WAP101, WAP108, WAP113...",17.587196,13.526260,0.951917,70.356851,0.664495,0.944625,0.996743
2,MAX,20,"WAP046, WAP084, WAP090, WAP102, WAP105, WAP116...",17.933808,13.978657,0.496442,72.255256,0.657980,0.960912,0.996743
3,AVERAGE,20,"WAP059, WAP091, WAP101, WAP103, WAP105, WAP107...",18.288905,12.355929,0.695355,124.481938,0.586319,0.960912,0.996743
4,MUTUAL_INFO,20,"WAP015, WAP051, WAP091, WAP102, WAP106, WAP108...",18.583199,13.622136,0.800666,83.000725,0.693811,0.973941,1.000000


In [25]:
# ============================================================================
# DATA PREPROCESSING
# ============================================================================

# 1. Filter Phase 2 data to include only configurations with finite TTS
print("\nFiltering Phase 2 data...")
phase2_finite = phase2_df[np.isfinite(phase2_df['tts_s'])].copy()
print(f"  Filtered rows: {len(phase2_finite)}")
print(f"  Removed rows with infinite TTS: {len(phase2_df) - len(phase2_finite)}")
print(f"  TTS range: {phase2_finite['tts_s'].min():.4f}s to {phase2_finite['tts_s'].max():.4f}s")

# 2. Filter pipeline data for k=20 only
print("\nPipeline: Filtering for k=20 configurations...")
k20_df = pipeline_df.copy()

# Filter to only show Entropy, Variance, Max, and Average
selected_methods = ['Entropy', 'Variance', 'Max', 'Average']
print(f"  Filtered methods: {k20_df['Importance_Method'].tolist()}")

# Convert to percentages
k20_df['Floor_Accuracy_Exact_Pct'] = k20_df['Floor_Accuracy_0'] * 100
k20_df['Floor_Accuracy_Plus1_Pct'] = k20_df['Floor_Accuracy_1'] * 100

print("\n✓ Data preprocessing complete")


Filtering Phase 2 data...
  Filtered rows: 144
  Removed rows with infinite TTS: 0
  TTS range: 0.1471s to 305.4221s

Pipeline: Filtering for k=20 configurations...
  Filtered methods: ['ENTROPY', 'VARIANCE', 'MAX', 'AVERAGE', 'MUTUAL_INFO']

✓ Data preprocessing complete


In [26]:
# ============================================================================
# DATA QUALITY REPORT AND SUMMARY STATISTICS
# ============================================================================

print("\n" + "="*80)
print("DATA QUALITY REPORT")
print("="*80)

# Phase 1 Summary
print("\n--- PHASE 1: QUBO PARAMETERS ---")
print(f"Configurations: {len(phase1_df)}")
print(f"Missing values:\n{phase1_df.isnull().sum()}")
print(f"\nPerformance Metrics:")
print(f"  Mean 3D Error: {phase1_df['mean_3d_error_m'].min():.2f}m (best) to {phase1_df['mean_3d_error_m'].max():.2f}m (worst)")
print(f"  Floor Accuracy: {phase1_df['floor_accuracy_0'].min():.2%} to {phase1_df['floor_accuracy_0'].max():.2%}")
print(f"\nBest configuration:")
best_phase1 = phase1_df.sort_values('mean_3d_error_m').iloc[0]
print(f"  k={int(best_phase1['k'])}, alpha={best_phase1['alpha']}, penalty={best_phase1['penalty']}")
print(f"  Mean Error: {best_phase1['mean_3d_error_m']:.2f}m")
print(f"  Floor Accuracy: {best_phase1['floor_accuracy_0']:.2%}")

# Phase 2 Summary
print("\n--- PHASE 2: OPENJIJ PARAMETERS ---")
print(f"Configurations (finite TTS): {len(phase2_finite)}")
print(f"Configurations (infinite TTS): {len(phase2_df) - len(phase2_finite)}")
print(f"\nPerformance Metrics:")
print(f"  TTS: {phase2_finite['tts_s'].min():.4f}s (best) to {phase2_finite['tts_s'].max():.4f}s (worst)")
print(f"  Success Rate: {phase2_finite['success_rate'].min():.2%} to {phase2_finite['success_rate'].max():.2%}")
print(f"  Mean 3D Error: {phase2_finite['mean_3d_error_m'].min():.2f}m to {phase2_finite['mean_3d_error_m'].max():.2f}m")
print(f"\nBest configuration (by TTS):")
best_phase2 = phase2_finite.sort_values('tts_s').iloc[0]
print(f"  sweeps={int(best_phase2['num_sweeps'])}, reads={int(best_phase2['num_reads'])}, beta={best_phase2['beta']}, gamma={best_phase2['gamma']}")
print(f"  TTS: {best_phase2['tts_s']:.4f}s, Success Rate: {best_phase2['success_rate']:.2%}")

# Importance Methods Summary
print("\n--- IMPORTANCE METHODS COMPARISON ---")
if len(k20_df) > 0:
    print(k20_df[['Importance_Method', 'Median_3D_Error_m', 'Floor_Accuracy_Exact_Pct']].to_string(index=False))
    print(f"\nBest method (by Median Error): {k20_df.sort_values('Median_3D_Error_m').iloc[0]['Importance_Method']}")
    print(f"Best method (by Floor Accuracy): {k20_df.sort_values('Floor_Accuracy_Exact_Pct', ascending=False).iloc[0]['Importance_Method']}")
else:
    print("WARNING: No data available for k=20 importance methods comparison")

print("\n" + "="*80)


DATA QUALITY REPORT

--- PHASE 1: QUBO PARAMETERS ---
Configurations: 96
Missing values:
k                    0
alpha                0
penalty              0
num_aps_selected     0
qubo_time_s          0
mean_3d_error_m      0
median_3d_error_m    0
floor_accuracy_0     0
floor_accuracy_1     0
floor_accuracy_2     0
dtype: int64

Performance Metrics:
  Mean 3D Error: 13.16m (best) to 27.58m (worst)
  Floor Accuracy: 51.14% to 83.06%

Best configuration:
  k=20, alpha=0.9, penalty=1.5
  Mean Error: 13.16m
  Floor Accuracy: 65.47%

--- PHASE 2: OPENJIJ PARAMETERS ---
Configurations (finite TTS): 144
Configurations (infinite TTS): 0

Performance Metrics:
  TTS: 0.1471s (best) to 305.4221s (worst)
  Success Rate: 2.00% to 100.00%
  Mean 3D Error: 14.40m to 39.22m

Best configuration (by TTS):
  sweeps=100, reads=10, beta=10.0, gamma=1.0
  TTS: 0.1471s, Success Rate: 100.00%

--- IMPORTANCE METHODS COMPARISON ---
Importance_Method  Median_3D_Error_m  Floor_Accuracy_Exact_Pct
          ENT

In [ ]:
k20_df.head()

,Importance_Method,Num_APs,Selected_APs,Mean_3D_Error_m,Median_3D_Error_m,Min_Error_m,Max_Error_m,Floor_Accuracy_0,Floor_Accuracy_1,Floor_Accuracy_2,Floor_Accuracy_Exact_Pct,Floor_Accuracy_Plus1_Pct
0,ENTROPY,20,"WAP016, WAP037, WAP090, WAP091, WAP103, WAP104...",16.997920,11.578227,0.206596,75.044313,0.661238,0.983713,1.000000,66.123779,98.371336
1,VARIANCE,20,"WAP089, WAP090, WAP091, WAP101, WAP108, WAP113...",17.587196,13.526260,0.951917,70.356851,0.664495,0.944625,0.996743,66.449511,94.462541
2,MAX,20,"WAP046, WAP084, WAP090, WAP102, WAP105, WAP116...",17.933808,13.978657,0.496442,72.255256,0.657980,0.960912,0.996743,65.798046,96.091205
3,AVERAGE,20,"WAP059, WAP091, WAP101, WAP103, WAP105, WAP107...",18.288905,12.355929,0.695355,124.481938,0.586319,0.960912,0.996743,58.631922,96.091205
4,MUTUAL_INFO,20,"WAP015, WAP051, WAP091, WAP102, WAP106, WAP108...",18.583199,13.622136,0.800666,83.000725,0.693811,0.973941,1.000000,69.381107,97.394137


: 